In [ ]:
#Öncelikle gerekli kütüphanelerimizi dahil edelim
import numpy as np
import pandas as pd 
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from argparse import Namespace
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
import missingno as msno 
from sklearn import ensemble
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, auc, roc_curve, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from matplotlib.legend_handler import HandlerLine2D
from warnings import filterwarnings
filterwarnings('ignore')

**Veri Setimizi Tanıma**

In [ ]:
tablet=pd.read_csv("../input/tabletpc-priceclassification/tablet.csv") #csv dosyamızı tablet isimli değişken tanımlayıp dahil edelim.
df = tablet.copy()

In [ ]:
df.head() #ilk 5 gözlemini görüntüleyelim.

In [ ]:
df.tail() #Veri çerçevemizin son 5 gözlemini görüntüleyelim.

In [ ]:
df.sample(5) #Veri çerçevemizin rasgele 5 gözlemini görüntüleyelim.

In [ ]:
df.shape #Veri çerçevemizin gözlem sayısının ve değişken tiplerinin kaç adet olduğunu gözlemleyelim.
#2000->gözlem sayısı ,20->değişken tipi 

In [ ]:
df["FiyatAraligi"].value_counts() 
#Değişkenler ve değerleri eşit olarak dağıldığı için dengeli bir dağılıma sahiptir.

In [ ]:
df.info() #Her bir değişkenimizin kaçar adet olduğunu,değişkenlerin tipini ve bellek kullanımını görüntüleyelim.

In [ ]:
df.isnull().sum().sum()  # Veri setimizde bulunan eksik verilerimizin sayısını görüntüleyelim. 

In [ ]:
df.isna().sum() #Veri çerçevemizdeki eksik değerlerini görüntüleyelim
#OnKameraMP ve RAM değişkenlerinde eksikler olduğu görülüyor.

In [ ]:
df.describe().T #Veri çerçevemizdeki sadece sayısal değişkenlerini ve temel istatistik değerlerini gözlemleyelim.

* Veri çerçevemizde en yüksek ortalamaya sahip RAM değişkenimizken, en düşük ortalamaya sahip Kalınlık değişkenimizdir.
* Çözünürlük yükseklik değişkenimizin medyanı ve ortalaması arasında önemli fark vardır.Değerleri birbirine çok yakın değildir diyebiliriz.
* Tabletlerin kalınlıkları benzer değerler diyebiliriz .Çünkü medyan ve ortalama farkı çok azdır.
* Aynı şekilde kalınlık değişkenimizde standart sapmamız en düşük olduğu için değerler daha düzenlidir sonucuna ulaşabiliriz.
* Ortalama ile standart sapma doğru orantılı olduğu için RAM değişkeninin standart sapması en büyüktür.
* RAM ,Batarya gucu ,çözünürlük yükseklik gibi değişkenlerin min ve max değerleri arasındaki fark oldukça fazladır.Bu değişkenlerimiz fiyat aralığını ve performansı yüksek derecede etkiler diyebilir miyiz ? 

In [ ]:
corr = df.corr() #Sayısal değerlerin aralarındaki ilişkiyi incelemek için korelasyon matrisi çizdirelim.
corr  

In [ ]:
sns.heatmap(corr,annot=True,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values); #Isı haritası ile daha rahat inceleyebiliriz.

!! 1 'e en yakın değer ArkaKameraMP ve OnKameraMP değişkeninin arasındadır.Bu ikili arasındaki ilişki en güçlü ve pozitiftir.  

**Değişkenleri Görselleştirme**

In [ ]:
sns.countplot(df["Dokunmatik"]);

In [ ]:
sns.boxplot(df["Agirlik"],color="salmon");

In [ ]:
sns.violinplot(df["RAM"],color="purple");

In [ ]:
sns.distplot(df["BataryaGucu"], bins = 20,color="red");

In [ ]:
sns.relplot(x="FiyatAraligi", y="BataryaGucu", kind="line", ci="sd",color="orange", data=df);
# Değişkenlerimizi tanırken de batarya gücünün tablet fiyatlarını etkilediğini düşünmüştük.Görüldüğü üzere fiyatla batarya gücü arasında doğru ve pozitif bir orantı var.

In [ ]:
sns.catplot(x="BataryaOmru",y="FiyatAraligi", jitter=False, data=df);

In [ ]:
sns.relplot(x="ArkaKameraMP", y="OnKameraMP", kind="line", ci="sd",color="green", data=df); #Doğrusal bir artış olduğunu görüyoruz.

In [ ]:
df["FiyatAraligi"].value_counts().plot.barh().set_title("Fiyat Aralığı değişkeninin sınıf frekansları");

In [ ]:
sns.relplot(x="FiyatAraligi", y="CekirdekSayisi", kind="line", ci="sd",color="purple", data=df);

In [ ]:
sns.boxplot(x="FiyatAraligi",y="ArkaKameraMP", palette="ch:.25",data=df); #ArkaKameraMP i fazla olan tabletlerin fiyatları daha pahalı olduğu sonucunu çıkartabiliriz.
#Bu değişkenin tablet alımında önemli bir rol olduğu söylenebilir.

In [ ]:
sns.barplot(x="FiyatAraligi",y="RAM",data=df); #Fiyatın pahalı olduğu yerlerde RAM değişkenimizin değeri daha büyüktür.
#Bu değişkenin tablet alımında önemli bir rol olduğu söylenebilir.

In [ ]:
sns.catplot(x="FiyatAraligi",y="DahiliBellek",kind="violin",palette="ch:.25",data=df); #40-60 arasında pahalı değişkenimiz daha genş bir gra

In [ ]:
sns.barplot(x="FiyatAraligi",y="BataryaOmru",data=df);

In [ ]:
sns.catplot(x="FiyatAraligi",y="CozunurlukYükseklik",kind="swarm",data=df); #Fiyat ve ÇozunurlukYukseklik arasında doğru orantı ilişki görüyoruz.

In [ ]:
(sns.
 FacetGrid(df, #bölerek göstermek için 
              hue="FiyatAraligi", #boyut katı eklenir
              height=5,
              xlim=(0,10000)) #x ekseni ayarlama
 .map(sns.kdeplot,"CozunurlukGenislik",shade=True)#yoğunluğu boyut olarak ekledik
 .add_legend()#kategori bilgilerini eklemek için
);

In [ ]:
sns.catplot(x="FiyatAraligi", y="DahiliBellek", hue="4G", kind="box", palette="ch:.25",data=df);

In [ ]:
sns.lineplot("FiyatAraligi" , "RAM", "MikroislemciHizi",  data=df);

In [ ]:
sns.catplot(x="FiyatAraligi", y="RAM", kind="boxen",hue="Dokunmatik",
            data=df.sort_values("FiyatAraligi")); #Dokunmatik olan ve olmayan fiyatı pahalı olan her tabletin RAM i de yüksektir diyebiliriz.

In [ ]:
sns.catplot(x="FiyatAraligi", y="MikroislemciHizi", kind="boxen",hue="CekirdekSayisi",
            data=df.sort_values("FiyatAraligi"));

**Veri Ön İşleme**

In [ ]:
#ordinal tanımlama(sıralanabilir)
#Bizim fiyat aralığı değişkenimiz sıralı verildiği için bu işleme gerek yok.Fakat kontrol amaçlı önem sırasına göre sıralayalım.
from pandas.api.types import CategoricalDtype

In [ ]:
df.FiyatAraligi.head()

In [ ]:
fiyat_kategoriler=["Çok Ucuz","Ucuz","Normal","Pahalı"]

In [ ]:
df.FiyatAraligi=df.FiyatAraligi.astype(CategoricalDtype(categories=fiyat_kategoriler, ordered=True))
df.FiyatAraligi.head()

In [ ]:
pip install missingno

In [ ]:
msno.matrix(df,figsize=(20, 10)); #Her beyaz çizgi eksik değeri gösteriyor. 
#İki değişkenimiz beyaz çizgiler içermektedir.

In [ ]:
df.drop("Renk", axis = 1, inplace = True)  #Renk değişkenimiz tabletin fiyat/performans açısından bir etkisi yoktur.Bu sepeble kaldırdık.

In [ ]:
df.head()

***Eksik Değerlerin Doldurulması***

Veri setimizi incelerken eksik gözlemlerin RAM ve OnKameraMP değişkenlerinde olduğunu görmüştük.

In [ ]:
df[df["RAM"].isnull()]  #RAM değişkeninin eksik olduğu yerler Fiyat Aralığının pahalı olduğu zamandadır.

In [ ]:
df.groupby("FiyatAraligi")[["RAM"]].mean()

In [ ]:
df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())]

In [ ]:
pahali_ram = df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())].index
pahali_ram #Fiyatı pahalı olup RAM değişkenimizde eksik olan değerlerin indexini görüntüledik.

In [ ]:
df.loc[pahali_ram ,"RAM"] = 3450  #Pahalı ->3449.35041 olduğu için en yakın olan değere 3450 yi atadık. 

In [ ]:
df[df["OnKameraMP"].isnull()] #OnKameraMP değişkeninin eksik olduğu yerler Fiyat Aralığının çok ucuz olduğu zamandadır.

In [ ]:
cokUcuz_onKameraMP = df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())].index
cokUcuz_onKameraMP #Fiyatı çok ucuz olup OnKameraMP değişkenimizde eksik olan değerlerin indexini görüntüledik.

In [ ]:
df.loc[cokUcuz_onKameraMP ,"OnKameraMP"] = 786 #Çok Ucuz ->785.31400 olduğu için en yakın olan 786 ye atadık.

**Eksik Verilerimizin Kalmadığını Kontrol Etmek**

In [ ]:
df.isna().sum()

In [ ]:
df["RAM"].unique()

In [ ]:
df["OnKameraMP"].unique()

Eksik değer içeren değişkenimiz kalmadı.Kategorik değişkenlerimizi sayısallaştırıp modelimizi eğitme aşamasına geçmeliyiz.

**Değişkenlerimizi Sayısallaştırma(Nominal)**

In [ ]:
label_encoder = preprocessing.LabelEncoder() #Label encoder kullanmamızın sebebi bu değişkenlerde sıralama işleminin uygulanamamasıdır.

In [ ]:
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth'])
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat'])
df['4G'] = label_encoder.fit_transform(df['4G'])
df['3G'] = label_encoder.fit_transform(df['3G'])
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik'])
df['WiFi'] = label_encoder.fit_transform(df['WiFi'])

In [ ]:
df.head()  #Bütün kategorik değişkenlerimizin değerleri sayısallaşmıştır.

**Modeli Eğitmek**

In [ ]:
X = df.drop("FiyatAraligi", axis = 1)
y = df["FiyatAraligi"] 

In [ ]:
y #Bağımlı değişkenimizi görüntüleyelim. (Fiyat Aralığı)

In [ ]:
X #Bağımsız değişkenimizi görüntüleyelim.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

In [ ]:
X_train #Gözlem sayısını görüntüleyelim.

In [ ]:
y_train #Gözlem sayısını görüntüleyelim.

In [ ]:
X_test #Gözlem sayısını görüntüleyelim.

In [ ]:
y_test #Gözlem sayısını görüntüleyelim.

**GaussianNB ile Eğitmek**

GaussianNB modeli yaratalım ve eğitimi gerçekleştirelim.

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)

In [ ]:
nb_model #Modelin parametrelerini görüntüleyelim.

In [ ]:
dir(nb_model) #Modelde kullanabileceğimiz komutları görüntüleyelim.

In [ ]:
X_test[0:10] #Modelimizin tahmin ettiği X_test gözlemlerinin ilk 10 tanesini gözlemleyelim.
nb_model.predict(X_test)[0:10]

In [ ]:
y_test[0:10] #Modelimizdeki ilk 10 gerçek veri değerlerimizi görüntüleyelim.

In [ ]:
y_pred = nb_model.predict(X_test) #Tüm X_test gözlemlerini tahmin ettirip y_pred isimli değişkene atayalım ve gösterelim.
y_pred

In [ ]:
y_test

Accuracy => Doğru tanımlanmış tüm vakaların ölçüsüdür. En çok tüm sınıflar eşit derecede önemli olduğunda kullanılır.

In [ ]:
accuracy_score(y_test, y_pred) #Doğruluk skorunu görüntüleyelim.
#Yüksek doğruluk varsa, modelimizin en iyisi olduğunu düşünebiliriz.
#Ancak performansımızı değerlendirmek için diğer parametrelere de bakmalıyız.
#Modelimiz %69 doğrudur.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)  #Karmaşıklık matrisini yazdıralım.

In [ ]:
(karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1]+karmasiklik_matrisi[2][2] + karmasiklik_matrisi[3][3]) / (karmasiklik_matrisi[0][0] + karmasiklik_matrisi[1][1] +  karmasiklik_matrisi[1][0] + karmasiklik_matrisi[0][1]+karmasiklik_matrisi[0][2]  + karmasiklik_matrisi[1][2]+karmasiklik_matrisi[0][3] + karmasiklik_matrisi[1][3]+karmasiklik_matrisi[2][1] + karmasiklik_matrisi[2][2]+karmasiklik_matrisi[2][3] + karmasiklik_matrisi[2][0]+karmasiklik_matrisi[3][0] + karmasiklik_matrisi[3][1]+karmasiklik_matrisi[3][2] + karmasiklik_matrisi[3][3])
#Karmaşıklık matris skorunu hesapladık.

In [ ]:
cross_val_score(nb_model, X_test, y_test, cv = 10) # 10 kez katlamalı çapraz doğrulama yapalım ,modelin skorunu tekrardan hesaplayalım.

In [ ]:
cross_val_score(nb_model, X_test, y_test, cv = 10).mean() #Ortalamalarını hesaplayalım.

In [ ]:
print(classification_report(y_test, y_pred)) #Sınıflandırma raporu ile yazdıralım.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore  #Precision skorunu görüntüleyelim.
#Precision değerinin yüksek olması bizim için model seçiminde önemli bir kriterdir.
#Modelimiz yaklaşık %77 doğrudur.Oldukça iyi hassasiyete sahibiz.

Recall ise Positive olarak tahmin etmemiz gereken işlemlerin ne kadarını Positive olarak tahmin ettiğimizi gösteren bir metriktir.

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore #Recall Skorunu görüntüleyelim.
#Yanlış Negatiflerin maliyeti yüksek olduğunda önemlidir.

F1 Skoru; recall ve precision skorunun ağırlıklı ortalamasıdır.Bu sebeple;
bu skor hem yanlış pozitifleri hem de yanlış negatifleri alır. 

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score #F1 skorunu görüntüleyelim.
# F1 skoru 1'deyken en iyi değerine ulaşır.
#Modelimizde F1 değerinde 1'e yakındır.

**Decision Tree ile Eğitmek**

In [ ]:
#cart = DecisionTreeClassifier(random_state = 42)
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

In [ ]:
cart_model #Modelin parametrelerini ve varsayılan değerleri görüntüleyelim.

In [ ]:
y_pred = cart_model.predict(X_test) #Model için parametreleri optimize etmeden tahmin ettirelim.

In [ ]:
accuracy_score(y_test, y_pred)  #X_test verileri için bulunduğu tahminlere göre doğruluk skorunu görüntüleyelim

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi) #Karmaşıklık matrisini yazdıralım.

Modelimizi iyileştirmek için parametreleri optimize edelim.

In [ ]:
cart_grid = {"max_depth": range(1,20),
            "min_samples_split" : range(2,100)}

In [ ]:
cart = DecisionTreeClassifier()
cart_cv = GridSearchCV(cart, cart_grid, cv = 3, n_jobs = -1, verbose = 2)
cart_cv_model = cart_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler : " + str(cart_cv_model.best_params_))
print("En iyi skor : " + str(cart_cv_model.best_score_))

In [ ]:
cart = DecisionTreeClassifier(max_depth = 7, min_samples_split =95)
cart_tuned = cart.fit(X_train, y_train)

In [ ]:
y_pred = cart_tuned.predict(X_test) 

In [ ]:
accuracy_score(y_test, y_pred) #Optimize ederek kurduğumuz model için tahminlerde bulunalım.
#Yüksek doğruluk varsa, modelimizin en iyisi olduğunu düşünebiliriz.
#Modelimiz yaklaşık %80 doğrudur.Bu skor modelimizin iyi olduğunu işaret eder.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred) #Karmaşıklık matrisini görüntüleyelim.
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(cart_tuned, X_test, y_test, cv = 10)

In [ ]:
cross_val_score(cart_tuned, X, y, cv = 10).mean() #10 skorun ortalamasına bakalım

In [ ]:
print(classification_report(y_test, y_pred)) #Modelimizin veriyi çok iyi kavradığını gözlemleyebiliyoruz.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore  #Precision skorunu görüntüleyelim.
#Precision değerinin yüksek olması bizim için model seçiminde önemli bir kriterdir.
#Modelimiz yaklaşık %93 doğrudur.Oldukça iyi hassasiyete sahibiz.

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore #Recall Skorunu görüntüleyelim.
#Duyarlılık değeri False Negative olarak tahminlemenin maliyetinin yüksek olduğu durumlarda bize yardımcı olacak bir metriktir. 
#Mümkün olduğunca yüksek olması gereklidir. Bizim modelimizde 0.8 ile yüksek değere sahip olduğunu gözlemliyoruz.

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score #F1 skorunu görüntüleyelim.
# F1 skoru 1'deyken en iyi değerine ulaşır.
#Modelimizde F1 değerinde 1'e yakındır.
#F1 skoru 0.8 bulunmuştur.İyi kavramış diyebiliriz.

Özniteliklerin karar ağacına göre önem sıralamasına bakalım.

In [ ]:
ranking = cart_tuned.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = X.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="purple", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()   #En çok RAM e önem verildiğini görmekteyiz.

**KNN ile Eğitmek & Komşu Sayısı Değişimini Gözlemlemek**

KNN modelini parametre optimizasyonu yaparak kuralım ve eğitelim.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)} # 2 ile 15 arasında olan her sayıyı komşu sayısı olarak deneyecek.

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(10)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)
#Doğruluk, modelinizin genel olarak ne kadar iyi performans gösterdiğini gösterir. Ancak, ayrıntılı bilgi vermez.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
score_list = [] #14 adet k değeri için doğruluk skorlarını görselleştirelim.

for each in range(2,15,1): # 2 den 15 e kadar olan komşu sayılarını görmek istiyoruz.
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    score_list.append(knn2.score(X_test, y_test))

plt.plot(range(2,15,1),score_list)
plt.xlabel("k en yakın komşu sayıları")
plt.ylabel("doğruluk skoru")
plt.show()

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10)

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10).mean()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

* 3 farklı modeli incelediğimizde en doğru modelin  knn modeli olduğunu söyleyebiliriz.Doğruluk skoru ,precision ,recall gibi değerleri diğer modellere göre karşılaştırdığımızda veriyi en iyi anlayan, tahmin eden model knn modelidir diyebiliriz.